In [1]:
import numpy as np
from keras import layers
from keras.layers import Input, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D
from keras.layers import AveragePooling2D, MaxPooling2D, Dropout, GlobalMaxPooling2D, GlobalAveragePooling2D
from keras.models import Model
from keras.preprocessing import image
from keras.utils import layer_utils
from keras.utils.data_utils import get_file
from keras.applications.imagenet_utils import preprocess_input
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
from keras.utils import plot_model
from keras.models import load_model

import keras.backend as K
K.set_image_data_format('channels_last')

import cv2

C:\Users\Joshua\Anaconda3\envs\SignModel\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
from sklearn.externals import joblib
from skimage.feature import hog

In [3]:
SignModel = load_model('signModel.h5')

In [4]:
path = 'img.jpg'
im = cv2.imread(path)
cv2.imshow('image',im)
cv2.waitKey(0)

-1

In [5]:
im_gray = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)
im_gray = cv2.GaussianBlur(im_gray, (5, 5), 0)

In [6]:
ret, im_th = cv2.threshold(im_gray, 90, 255, cv2.THRESH_BINARY_INV)


In [7]:
_, ctrs, hier = cv2.findContours(im_th.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

In [8]:
rects = [cv2.boundingRect(ctr) for ctr in ctrs]
print(rects)

[(17, 4031, 1, 1), (0, 3988, 17, 44), (0, 3974, 5, 2), (1442, 2661, 153, 432), (960, 2048, 29, 408), (1875, 2025, 193, 331), (811, 2015, 149, 198), (1259, 1330, 199, 318), (2438, 1143, 171, 397), (745, 500, 191, 468), (1595, 473, 214, 410)]


In [9]:
for rect in rects:
    cv2.rectangle(im, (rect[0], rect[1]), (rect[0] + rect[2], rect[1] + rect[3]), (0, 255, 0), 3)
    leng = int(rect[3] * 1.6)
    pt1 = int(rect[1] + rect[3] // 2 - leng // 2)
    print(pt1)
    pt2 = int(rect[0] + rect[2] // 2 - leng // 2)
    roi = im_th[pt1:pt1+leng, pt2:pt2+leng]
    roi = cv2.resize(roi, (28, 28), interpolation=cv2.INTER_AREA)
    roi = cv2.dilate(roi, (3, 3))
    print(roi)
    roi_hog_fd = hog(roi, orientations=9, pixels_per_cell=(14, 14), cells_per_block=(1, 1), visualise=False)
    print(roi_hog_fd)
    print(SignModel.predict(np.array([roi_hog_fd], 'float64')))
    nbr = SignModel.predict(np.array([roi_hog_fd], 'float64'))
    cv2.putText(im, str(int(nbr[0])), (rect[0], rect[1]),cv2.FONT_HERSHEY_DUPLEX, 2, (0, 255, 255), 3)

cv2.imshow("Resulting Image with Rectangular ROIs", im)
cv2.waitKey()

4031
[[255 255 255 255 255 255 255 255 255 255 255 255 255 255 255 255 255 255
  255 255 255 255 255 255 255 255 255 255]
 [255 255 255 255 255 255 255 255 255 255 255 255 255 255 255 255 255 255
  255 255 255 255 255 255 255 255 255 255]
 [255 255 255 255 255 255 255 255 255 255 255 255 255 255 255 255 255 255
  255 255 255 255 255 255 255 255 255 255]
 [255 255 255 255 255 255 255 255 255 255 255 255 255 255 255 255 255 255
  255 255 255 255 255 255 255 255 255 255]
 [255 255 255 255 255 255 255 255 255 255 255 255 255 255 255 255 255 255
  255 255 255 255 255 255 255 255 255 255]
 [255 255 255 255 255 255 255 255 255 255 255 255 255 255 255 255 255 255
  255 255 255 255 255 255 255 255 255 255]
 [255 255 255 255 255 255 255 255 255 255 255 255 255 255 255 255 255 255
  255 255 255 255 255 255 255 255 255 255]
 [255 255 255 255 255 255 255 255 255 255 255 255 255 255 255 255 255 255
  255 255 255 255 255 255 255 255 255 255]
 [255 255 255 255 255 255 255 255 255 255 255 255 255 255 2

C:\Users\Joshua\Anaconda3\envs\SignModel\lib\site-packages\skimage\feature\_hog.py:119: skimage_deprecation: Default value of `block_norm`==`L1` is deprecated and will be changed to `L2-Hys` in v0.15
  'be changed to `L2-Hys` in v0.15', skimage_deprecation)


ValueError: Error when checking : expected input_1 to have 4 dimensions, but got array with shape (1, 36)

In [27]:
def overlap(x1,y1,w1,h1,x2,y2,w2,h2):
    if x1 < x2 and x1+w1 > x2 and y1 < y2 and y1+h1 > y2:
        return True
    elif x1 > x2 and x1 < x2+w2 and x2+w2 < x1+x2 and y1 < y2 and y1+h1 > y2:
        return True
    elif x1 < x2 and x1+w1 > x2 and y1 > y2 and y1 < y2+h2 and y2+h2 < y1+h1:
        return True
    elif x1 > x2 and x1 < x2+w2 and x2+w2 < x1+w1 and y1 > y2 and y1 < y2+h2 and y2+h2 < y1+h1:
        return True
    else:
        return False

im = cv2.imread('img.jpg')
print(im.shape) #check if the image is loaded correctly
imgray = cv2.cvtColor(im,cv2.COLOR_BGR2GRAY)
imgray = cv2.GaussianBlur(imgray, (5, 5), 0)
ret,thresh = cv2.threshold(imgray,90,255,cv2.THRESH_BINARY_INV)
_,contours, hierarchy = cv2.findContours(thresh,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
rectlist = []
for cont in contours:
    x,y,w,h = cv2.boundingRect(cont)
    if w*h>50*50:
        rectlist.append((x,y,w,h)) 
for i in range(len(rectlist)):
    for j in range(len(rectlist[i:])-1):
        if overlap(rectlist[i][0],rectlist[i][1],rectlist[i][2],rectlist[i][3],rectlist[i:][j][0],rectlist[i:][j][1],rectlist[i:][j][2],rectlist[i:][j][3]):
            if rectlist[i][2]*rectlist[i][3] > rectlist[i:][j][2]*rectlist[i:][j][3]:
                del rectlist[i+j]
            else:
                del rectlist[i]
print(rectlist)
for rect in rectlist:
    crop_img = im[rect[1]:rect[1]+rect[3], rect[0]:rect[0]+rect[2]]
    cv2.imshow("cropped", crop_img)
    cv2.waitKey(0)
                

cv2.imshow("window title", im)
cv2.waitKey()


    

(4032, 3024, 3)
[(1442, 2661, 153, 432), (960, 2048, 29, 408), (1875, 2025, 193, 331), (811, 2015, 149, 198), (1259, 1330, 199, 318), (2438, 1143, 171, 397), (2449, 1151, 149, 201), (745, 500, 191, 468), (1595, 473, 214, 410)]


-1

In [5]:
import numpy as np
from keras import layers
from keras.layers import Input, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D
from keras.layers import AveragePooling2D, MaxPooling2D, Dropout, GlobalMaxPooling2D, GlobalAveragePooling2D
from keras.models import Model
from keras.preprocessing import image
from keras.utils import layer_utils
from keras.utils.data_utils import get_file
from keras.applications.imagenet_utils import preprocess_input
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
from keras.utils import plot_model
from keras.models import load_model

import keras.backend as K
K.set_image_data_format('channels_last')

import cv2

from sklearn.externals import joblib
from skimage.feature import hog

SignModel = load_model('signModel.h5')

def overlap(x1, y1, w1, h1, x2, y2, w2, h2):
    if x1 < x2 and x1 + w1 > x2 and y1 < y2 and y1 + h1 > y2:
        return True
    elif x1 > x2 and x1 < x2 + w2 and x2 + w2 < x1 + x2 and y1 < y2 and y1 + h1 > y2:
        return True
    elif x1 < x2 and x1 + w1 > x2 and y1 > y2 and y1 < y2 + h2 and y2 + h2 < y1 + h1:
        return True
    elif x1 > x2 and x1 < x2 + w2 and x2 + w2 < x1 + w1 and y1 > y2 and y1 < y2 + h2 and y2 + h2 < y1 + h1:
        return True
    else:
        return False


im = cv2.imread('img.jpg')
imgray = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)
imgray = cv2.GaussianBlur(imgray, (5, 5), 0)
cv2.waitKey()
ret, thresh = cv2.threshold(imgray, 90, 255, cv2.THRESH_BINARY_INV)
im2, contours, hierarchy = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
rectlist = []
for cont in contours:
    x, y, w, h = cv2.boundingRect(cont)
    if w * h > 50 * 50:
        rectlist.append((x, y, w, h))

temp = list(rectlist)
for i in range(len(rectlist)):
    for j in range(len(rectlist[i:]) - 1):
        if overlap(rectlist[i][0], rectlist[i][1], rectlist[i][2], rectlist[i][3], rectlist[i:][j][0],
                   rectlist[i:][j][1], rectlist[i:][j][2], rectlist[i:][j][3]):
            if rectlist[i][2] * rectlist[i][3] > rectlist[i:][j][2] * rectlist[i:][j][3]:
                temp.remove(rectlist[i + j])
            else:
                temp.remove(rectlist[i])
rectlist = temp
for rect in rectlist:
    crop_imgOrig = im2[rect[1]:rect[1] + rect[3], rect[0]:rect[0] + rect[2]]
    crop_img = cv2.dilate(crop_imgOrig,(150,150),iterations=30)
    image = cv2.copyMakeBorder(cv2.resize(crop_img, (16, 16)), 6, 6, 6, 6, cv2.BORDER_CONSTANT)
    prediction = SignModel.predict(np.array(image).reshape(-1,28,28,1))
    print(np.where(prediction[0]>0.5)[0][0])
    cv2.imshow("orginal",im[rect[1]:rect[1] + rect[3], rect[0]:rect[0] + rect[2]])
    cv2.waitKey(0)

3
1
